<a href="https://colab.research.google.com/github/aeerema/aeerema.github.io/blob/main/queue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pytelegrambotapi

In [ ]:
import telebot
from telebot import types
import time
bot = telebot.TeleBot("5889436124:AAHpMAN2ZbI-PjFrEMskYF-KI5SJgGxptZc")

In [ ]:
from enum import Enum

class States(Enum):
    START = 0
    ROLE = 1
    NAME = 2
    TICKET = 3
    QUEUE = 4
    EXAM = 5
    SHABAT = 9

class Answers(Enum):
    STUDENT = "Я студент"
    TEACHER = "Я преподаватель"
    TICKET = "Взять талончик!"
    INFO = "Информация об очереди"
    NEXTPLS = "Вызвать следующего"

queue_is_empty = True

In [ ]:
import sqlite3
from sqlite3 import Error

def create_markup(*button_names):
    num_of_buttons = len(button_names)
    if num_of_buttons == 0:
        return types.ReplyKeyboardRemove()
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    for button_name in button_names:
        keyboard.add(types.KeyboardButton(button_name))
    return keyboard

def connect_to_DB():
    connection = None
    try:
        connection = sqlite3.connect("DB.sqlite", check_same_thread=False)
        print("Соединение с базой установлено успешно")
    except Error as error_info:
        print(f"Ошибка исполнения запроса: '{error_info}'")

    return connection

def execute_query(query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
    except Error as error_info:
        print(f"Ошибка исполнения запроса: '{error_info}'")

def execute_read_query(query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as error_info:
        print(f"Ошибка исполнения запроса: '{error_info}'")


def add_user(user_id):
    ids = execute_read_query("SELECT id from users")
    if not (user_id,) in ids:
        query = f"INSERT INTO users (id, state, attempt) VALUES ({user_id}, {States.START.value}, 0)"
        execute_query(query) 

def set_name(user_id, name):
    query = f"UPDATE users SET name='{name}' WHERE id={user_id}"
    execute_query(query)

def get_name(user_id):
    query_result = execute_read_query(f"SELECT name from users WHERE id={user_id}")
    return query_result[0][0] if query_result else "Безымянный"

def set_state(user_id, state):
    execute_query(f"UPDATE users SET state={state} WHERE id={user_id}")

def get_state(user_id):
    query_result = execute_read_query(f"SELECT state from users WHERE id={user_id}")
    return query_result[0][0] if query_result else 0

def set_ticket(user_id):
    global queue_is_empty
    
    attempt = execute_read_query(f"SELECT attempt from users WHERE id={user_id}")[0][0]
    query_result = execute_read_query(f"SELECT id from users WHERE state={States.EXAM.value}")
    
    if queue_is_empty and not query_result:
        send_to_exam(user_id)
        return False
    else: 
        queue_is_empty = False
        query_result = execute_read_query(f"SELECT MAX(ticket) from users "\
                                          f"WHERE attempt={attempt} AND state={States.QUEUE.value}")
        max_ticket = query_result[0][0] if query_result else 0
        if max_ticket == None:
            max_ticket = 0
        set_state(user_id, States.QUEUE.value)
        execute_query(f"UPDATE users SET ticket={max_ticket+1}, "\
                      f"score={max_ticket + attempt*20} WHERE id={user_id}")
        return True
        

def get_ticket(user_id):
    query_result = execute_read_query(f"SELECT attempt, ticket from users WHERE id={user_id}")
    if query_result:
        attempt = query_result[0][0]
        ticket = query_result[0][1]
        return f"#{attempt}-{ticket}"
    return "None"

def send_to_exam(user_id):
    attempt = execute_read_query(f"SELECT attempt from users WHERE id={user_id}")[0][0]+1
    execute_query(f"UPDATE users SET ticket=NULL WHERE id={user_id}")
    execute_query(f"UPDATE users SET attempt={attempt} WHERE id={user_id}")
    execute_query(f"UPDATE users SET state={States.EXAM.value} WHERE id={user_id}")
    execute_query(f"UPDATE users SET score=NULL WHERE id={user_id}")
    # time
    execute_query(f"INSERT INTO times (id, attempt, start, duration) "\
                  f"VALUES ({user_id}, {attempt}, {get_time()}, NULL)")

    try:
        bot.send_message(user_id, "Твоя очередь!", 
                     reply_markup = create_markup())
    except:
        pass
    student_name = execute_read_query(f"SELECT name FROM users WHERE id={user_id}")[0][0]
    teachet_id = execute_read_query(f"SELECT id FROM users WHERE state={States.SHABAT.value}")[0][0]
    bot.send_message(teachet_id, f"К вам подходит студент {student_name}")
    print(f"Пользователь {student_name} идет к Шабату.")

def get_next():
    global queue_is_empty
    
    query_result = execute_read_query(f"SELECT id, attempt from users WHERE state={States.EXAM.value}")
    if query_result:
        last_id = query_result[0][0]
        attempt = query_result[0][1]
        execute_query(f"UPDATE users SET state={States.TICKET.value} WHERE id={last_id}")
        execute_query(f"UPDATE users SET score=NULL WHERE id={last_id}")
        laststart = execute_read_query(f"SELECT start from times "\
                                       f"WHERE id={last_id} AND attempt={attempt}")[0][0]
        execute_query(f"UPDATE times SET duration={get_time() - laststart} "\
                      f"WHERE id={last_id} AND attempt={attempt}")
        try:
            bot.send_message(last_id, 
                             "Попытка не пытка, как будешь готов попробуй еще раз!", 
                             reply_markup = create_markup(Answers.INFO.value, 
                                                          Answers.TICKET.value))
        except:
            pass
    if not queue_is_empty:
        query_result = execute_read_query(f"SELECT id from users "\
                                          f"WHERE state={States.QUEUE.value} "\
                                          f"ORDER BY score limit 4")
        send_to_exam(query_result[0][0])
        top_len = len(query_result)-1
        for i in range(top_len):
            try:
                bot.send_message(query_result[i+1][0], get_info(query_result[i+1][0]))
                print(f"Пользователю {get_name(query_result[i+1][0])} "\
                      f"отправили сообщение ТОП3 и он ее получил.")
            except:
                print(f"Пользователю {get_name(query_result[i+1][0])} "\
                      f"отправили сообщение ТОП3, но он ее не получил.")
            
        if not execute_read_query(f"SELECT id from users WHERE state={States.QUEUE.value}"):
            queue_is_empty = True
        return False
    return True
        
    
def get_info(user_id):
    query_result = execute_read_query(f"SELECT Count(*)-1 FROM users "\
                                      f"WHERE state = {States.QUEUE.value} AND "\
                                      f"score <= (SELECT score FROM users "\
                                      f"WHERE id = {user_id})")
    num_before = query_result[0][0] if query_result else 0
    result = f"Перед тобой около {num_before} человек(а) в очереди.\n"
    avg_time = execute_read_query("SELECT AVG(duration) FROM times "\
                                  f"WHERE duration IS NOT NULL")
    avg_time = avg_time[0][0] if avg_time else 0
    avg_time = 0 if avg_time == None else int(avg_time)
    total_time = (num_before+1) * avg_time
    return (result+f"Среднее время прохождение через Шабата ~ "\
            f"{avg_time//60:02d}:{avg_time%60:02d}\n"\
            f"Тебе осталось ждать примерно {total_time//60:02d}:{total_time%60:02d}")
#             f"{avg_time//360:02d}:{avg_time//60%60:02d}:{avg_time%360:02d}\n"\
#             f"Тебе осталось ждать примерно "\
#             f"{total_time//360:02d}:{total_time//60%60:02d}:{total_time%360:02d}")

def get_info_lite(user_id):
    attempt = execute_read_query(f"SELECT attempt from users WHERE id={user_id}")[0][0]
    query_result = execute_read_query(f"SELECT MAX(ticket) from users "\
                                          f"WHERE attempt={attempt} AND state={States.QUEUE.value}")
    max_ticket = query_result[0][0] if query_result else 0
    ticket = 0 if max_ticket == None else max_ticket + 1
    score = ticket + attempt*20
    
    query_result = execute_read_query(f"SELECT Count(*) FROM users "\
                                      f"WHERE state = {States.QUEUE.value} AND "\
                                      f"score <= {score}")
    num_before = query_result[0][0] if query_result else 0
    result = f"Если сейчас встанешь в очередь, то перед тобой будет около {num_before} человек(а).\n"
    avg_time = execute_read_query("SELECT AVG(duration) FROM times "\
                                  f"WHERE duration IS NOT NULL")
    avg_time = avg_time[0][0] if avg_time else 0
    avg_time = 0 if avg_time == None else int(avg_time)
    total_time = (num_before+1) * avg_time
    return (result+f"Среднее время прохождение через Шабата ~ "\
            f"{avg_time//60:02d}:{avg_time%60:02d}\n"\
            f"Тебе осталось ждать примерно {total_time//60:02d}:{total_time%60:02d}")
#             f"{avg_time//360:02d}:{avg_time//60%60:02d}:{avg_time%360:02d}\n"\
#             f"Тебе осталось ждать примерно "\
#             f"{total_time//360:02d}:{total_time//60%60:02d}:{total_time%360:02d}")
    
        
def get_time():
    tm = time.localtime()
    return tm.tm_hour*60 + tm.tm_min
#     return tm.tm_hour*60*60 + tm.tm_min*60 + tm.tm_sec

In [ ]:
connection = connect_to_DB()
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    state INTEGER,
    name TEXT,
    attempt INTEGER,
    ticket INTEGER,
    score INTEGER
);
"""
execute_query(create_users_table)

create_times_table = """
CREATE TABLE IF NOT EXISTS times (
    id INTEGER,
    attempt INTEGER,
    start INTEGER,
    duration INTEGER,
    PRIMARY KEY (id, attempt)
);
"""
execute_query(create_times_table)

In [ ]:
@bot.message_handler(commands=["start"])
def cmd_start(message):
    user_id = message.chat.id
    add_user(user_id)
    set_state(user_id, States.ROLE.value)
    
    bot.send_message(user_id, 
                     "Привет! Я попробую помочь тебе не потеряться в очереди на экзамене по математике. "\
                     "Для начала давай определимся, кто ты?",  
                     reply_markup = create_markup(Answers.STUDENT.value, 
                                                  Answers.TEACHER.value))
    
    print(f"Пользователь id_{user_id} внесен в базу.")

# @bot.message_handler(commands=["reset"])
# def cmd_reset(message):
#     bot.send_message(message.chat.id, 
#                      "Что ж, начнём по-новой. Какова твоя роль на экзамене?",  
#                      reply_markup = student_teacher_markup)
#     set_state(message.chat.id, States.ROLE.value)

@bot.message_handler(func=lambda message: get_state(message.chat.id) == States.ROLE.value)
def user_entered_role(message):
    user_id = message.chat.id
    
    if message.text.strip() == Answers.STUDENT.value:
        set_state(user_id, States.NAME.value)
        
        bot.send_message(user_id, 
                         "Отлично, теперь введи, пожалуйста, имя. Помни, что Шабат будет его видеть!", 
                         reply_markup = create_markup())
        
        print(f"Пользователю id_{user_id} ставиться роль студент.")
        
    elif message.text.strip() == Answers.TEACHER.value:
        
        set_state(user_id, States.SHABAT.value)
        set_name(user_id, "Шабат Георгий Борисович")
        
        bot.send_message(user_id, 
                         "Добрый день, Георгий Борисович!", 
                         reply_markup = create_markup(Answers.NEXTPLS.value))
        
        print(f"Пользователю id_{user_id} ставиться роль преподаватель.")
        
    else:
        bot.send_message(user_id, "Выберите одну из кнопок.")


@bot.message_handler(func=lambda message: get_state(message.chat.id) == States.NAME.value)
def user_entered_name(message):
    user_id = message.chat.id 
    set_name(user_id, message.text.strip())
    set_state(user_id, States.TICKET.value)
    
    bot.send_message(user_id, 
                     "Отлично, теперь можно встать в очередь!", 
                     reply_markup = create_markup(Answers.INFO.value, 
                                                  Answers.TICKET.value))
    
    print(f"Пользователь id_{user_id} зарегестририровался как {get_name(user_id)}.")

@bot.message_handler(func=lambda message: get_state(message.chat.id) == States.TICKET.value)
def user_wants_ticket(message):
    user_id = message.chat.id
    message_text = message.text.strip()
    
    if message_text == Answers.TICKET.value:
        if set_ticket(user_id):
            bot.send_message(user_id, 
                         f"Твой талончик: {get_ticket(user_id)}", 
                         reply_markup = create_markup(Answers.INFO.value))
            print(f"Пользователь {get_name(user_id)} получает талон {get_ticket(user_id)}.")
        else:
            print(f"Пользователь {get_name(user_id)} без получения талона сразу идет к Шабату.")
    elif message_text == Answers.INFO.value:
        bot.send_message(user_id, get_info_lite(user_id))
        print(f"Пользователь {get_name(user_id)} (не в очереди) запросил инфо и получил ее.")
    else:
        bot.send_message(user_id, "Выберите одну из кнопок.")
        

@bot.message_handler(func=lambda message: get_state(message.chat.id) == States.QUEUE.value)
def user_in_queue(message):
    user_id = message.chat.id
    message_text = message.text.strip()
    
    if message_text == Answers.INFO.value:
        bot.send_message(user_id, get_info(user_id))
        print(f"Пользователь {get_name(user_id)} запросил инфо и получил ее.")


@bot.message_handler(func=lambda message: get_state(message.chat.id) == States.SHABAT.value)
def shabat(message):
    user_id = message.chat.id
    message_text = message.text.strip()
    if message_text == Answers.NEXTPLS.value:
        bot.delete_message(user_id, message.message_id)
        if get_next():
            bot.send_message(user_id, "К сожалению, очередь сейчас пуста.")
            print(f"Шабат вызвал следующего, но очередь пуста.")

In [ ]:
bot.polling()